# NRL Feature Engineering Pipeline

This notebook transforms the raw NRL match data into a feature-rich, model-ready dataset. It follows a structured, multi-step process to engineer features related to team form, strength, and match context, while carefully preventing data leakage.

**Objective:** To create a comprehensive `nrl_matches_final_model_ready.csv` file that will serve as the input for our machine learning models.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## Step 1: Foundational Data Cleaning & Setup

This is the most critical first step. We perform essential cleaning and setup tasks:
- **Load Data**: Ingest the raw CSV file.
- **Date Conversion**: Convert the 'Date' column to a proper datetime format.
- **Chronological Sort**: Sort the entire dataset by date. **This is crucial for all time-series feature engineering** to prevent looking into the future.
- **Create Target Variable**: Engineer the `Home_Win` binary target variable.
- **Create Margin**: Calculate the `Home_Margin` for performance analysis.

In [ ]:
def load_and_clean_nrl_data(filepath='data/nrlBaselineDataWithWeather.csv'):
    """Load, clean, and sort the foundational NRL dataset."""
    print("--- Step 1: Loading & Cleaning Data ---")
    try:
        df = pd.read_csv(filepath)
        print(f"✓ Loaded dataset: {filepath}")
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    
    # Date Conversion and Sorting
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df = df.sort_values(by='Date').reset_index(drop=True)
    
    # Create Target and Margin
    df['Home_Win'] = (df['Home Score'] > df['Away Score']).astype(int)
    home_win_rate = df['Home_Win'].mean()
    df['Home_Margin'] = df['Home Score'] - df['Away Score']
    df['match_id'] = df.index

    print(f"✓ Data cleaned and sorted. Shape: {df.shape}")
    print(f"✓ Date range: {df['Date'].min().strftime('%Y-%m-%d')} to {df['Date'].max().strftime('%Y-%m-%d')}")
    print(f"Total matches: {len(df)}")
    print(f"Unique teams: {len(set(df['Home Team'].unique()) | set(df['Away Team'].unique()))}")
    print(f"Missing values per column:")
    for col in df.columns:
        missing = df[col].isnull().sum()
        if missing > 0:
            print(f"  {col}: {missing} ({missing/len(df)*100:.1f}%)")
    
    print(f"\nHome team advantages:")
    print(f"  Win rate: {home_win_rate:.3f}")
    print(f"  Average margin: {df['Home_Margin'].mean():.2f}")
    
    return df


def preview_data(df, n_rows=5):
    """
    Preview the cleaned dataset
    """
    print(f"\n=== DATA PREVIEW (First {n_rows} rows) ===")
    key_columns = ['Date', 'Home Team', 'Away Team', 'Home Score', 'Away Score', 
                   'Home_Win', 'Home_Margin', 'match_id', 'temperature_category']
    print(df[key_columns].head(n_rows).to_string(index=False))
    
    print(f"\n=== DATA TYPES ===")
    print(df[key_columns].dtypes)

# Execute Step 1
df_cleaned = load_and_clean_nrl_data()
if df_cleaned is not None:
    preview_data(df_cleaned)

## Step 2: Create Team-Level Stats DataFrame

To calculate rolling statistics for each team, we need to transform the data from a *match-centric* view to a *team-centric* view. We "melt" the DataFrame so that each match is represented by two rows: one for the home team and one for the away team.

This structure makes it trivial to perform `groupby('team_name').rolling(...)` operations in the next step.

In [ ]:
def create_team_level_stats(df):
    """Transform match-level data to team-level data."""
    print("\n--- Step 2: Creating Team-Level Stats ---")
    home_df = df[['match_id', 'Date', 'Home Team', 'Home Score', 'Away Score', 'Home_Win']].copy()
    home_df.rename(columns={'Home Team': 'team_name', 'Home Score': 'points_for', 'Away Score': 'points_against', 'Home_Win': 'won'}, inplace=True)
    home_df['is_home'] = 1
    home_df['opponent'] = df['Away Team']

    away_df = df[['match_id', 'Date', 'Away Team', 'Home Score', 'Away Score', 'Home_Win']].copy()
    away_df.rename(columns={'Away Team': 'team_name', 'Away Score': 'points_for', 'Home Score': 'points_against'}, inplace=True)
    away_df['won'] = 1 - away_df['Home_Win']
    away_df['is_home'] = 0
    away_df['opponent'] = df['Home Team']
    away_df = away_df.drop(columns=['Home_Win'])

    team_stats_df = pd.concat([home_df, away_df], ignore_index=True)
    team_stats_df = team_stats_df.sort_values(['Date', 'team_name']).reset_index(drop=True)
    team_stats_df['margin'] = team_stats_df['points_for'] - team_stats_df['points_against']
    team_stats_df['lost'] = 1 - team_stats_df['won']
    
    print(f"✓ Team-level data created. Shape: {team_stats_df.shape}")
    return team_stats_df

def preview_team_level_stats(team_stats_df, team_name=None, n_rows=10):
        # Data validation and summary
    print(f" Original matches: {len(df)}")
    print(f" Team records created: {len(team_stats_df)} (should be 2x matches)")
    print(f" Unique teams: {team_stats_df['team_name'].nunique()}")
    print(f" Date range: {team_stats_df['Date'].min().strftime('%Y-%m-%d')} to {team_stats_df['Date'].max().strftime('%Y-%m-%d')}")
    
    # Team performance summary
    team_summary = team_stats_df.groupby('team_name').agg({
        'won': ['count', 'sum', 'mean'],
        'points_for': 'mean',
        'points_against': 'mean',
        'margin': 'mean'
    }).round(3)
    
    team_summary.columns = ['Games_Played', 'Wins', 'Win_Rate', 'Avg_Points_For', 'Avg_Points_Against', 'Avg_Margin']
    team_summary = team_summary.sort_values('Win_Rate', ascending=False)
    
    print(f"\n=== TEAM PERFORMANCE SUMMARY ===")
    print("Top 5 teams by win rate:")
    print(team_summary.head().to_string())
    
    print(f"\nBottom 5 teams by win rate:")
    print(team_summary.tail().to_string())
    
    # Home vs Away performance
    home_away_stats = team_stats_df.groupby('is_home').agg({
        'won': 'mean',
        'points_for': 'mean',
        'points_against': 'mean',
        'margin': 'mean'
    }).round(3)
    
    home_away_stats.index = ['Away', 'Home']
    print(f"\n=== HOME vs AWAY ADVANTAGE ===")
    print(home_away_stats.to_string())

    if team_name:
        preview_df = team_stats_df[team_stats_df['team_name'] == team_name].head(n_rows)
        print(f"\n=== TEAM STATS PREVIEW: {team_name} (First {n_rows} games) ===")
    else:
        preview_df = team_stats_df.head(n_rows)
        print(f"\n=== TEAM STATS PREVIEW (First {n_rows} rows) ===")
    
    key_columns = ['Date', 'team_name', 'is_home', 'opponent', 'points_for', 
                   'points_against', 'margin', 'won']
    
    print(preview_df[key_columns].to_string(index=False))
    
    print(f"\n=== TEAM STATS DATA TYPES ===")
    print(team_stats_df[key_columns].dtypes)

# Execute Step 2
team_stats_df = create_team_level_stats(df_cleaned)
if team_stats_df is not None:
    preview_team_level_stats(team_stats_df)

## Step 3: Calculate Rolling "Form" Features

This is where we quantify each team's recent performance, or "form". We use rolling windows to calculate moving averages for key metrics.


**Crucial for preventing data leakage:** We use `.shift(1)` after every rolling calculation. This ensures that the features for a given match are calculated using data from *previous* matches only.

In [ ]:
def calculate_rolling_features(team_stats_df):
    """Calculate rolling averages and streaks for each team."""
    print("\n--- Step 3: Calculating Rolling Form Features ---")
    df = team_stats_df.copy().sort_values(['team_name', 'Date'])
    windows = [3, 5, 8]

    for window in windows:
        df[f'rolling_avg_margin_{window}'] = df.groupby('team_name')['margin'].transform(lambda x: x.rolling(window, 1).mean().shift(1))
        df[f'rolling_win_percentage_{window}'] = df.groupby('team_name')['won'].transform(lambda x: x.rolling(window, 1).mean().shift(1))
        df[f'rolling_avg_points_for_{window}'] = df.groupby('team_name')['points_for'].transform(lambda x: x.rolling(window, 1).mean().shift(1))
        df[f'rolling_avg_points_against_{window}'] = df.groupby('team_name')['points_against'].transform(lambda x: x.rolling(window, 1).mean().shift(1))
        
    print(f"✓ Rolling features calculated for windows: {windows}")

    # Recent form (last 3 games) - more granular
    df['recent_wins_3'] = (
        df.groupby('team_name')['won']
        .rolling(window=3, min_periods=1)
        .sum()
        .shift(1)
        .reset_index(level=0, drop=True)
    )
    
    # Games since last win/loss (simplified approach to avoid index issues)
    df['games_since_win'] = 0
    df['games_since_loss'] = 0
    
    for team in df['team_name'].unique():
        team_mask = df['team_name'] == team
        team_data = df[team_mask].copy()
        team_data = team_data.sort_values('Date')
        
        games_since_win = []
        games_since_loss = []
        
        for i in range(len(team_data)):
            if i == 0:
                games_since_win.append(0)
                games_since_loss.append(0)
                continue
            
            # Count games since last win
            win_count = 0
            win_found = False
            for j in range(i-1, -1, -1):
                if team_data.iloc[j]['won'] == 1:
                    win_found = True
                    break
                win_count += 1
            games_since_win.append(win_count if win_found else i)
            
            # Count games since last loss
            loss_count = 0
            loss_found = False
            for j in range(i-1, -1, -1):
                if team_data.iloc[j]['won'] == 0:
                    loss_found = True
                    break
                loss_count += 1
            games_since_loss.append(loss_count if loss_found else i)
        
        df.loc[team_mask, 'games_since_win'] = games_since_win
        df.loc[team_mask, 'games_since_loss'] = games_since_loss

    return df

# Execute Step 3
team_stats_form = calculate_rolling_features(team_stats_df)

streaks features

In [ ]:
def calculate_streaks(df):
    """
    Calculate winning and losing streaks for each team
    """
    
    def get_current_streak(series):
        """Calculate current win/loss streak from a boolean series"""
        if len(series) == 0:
            return 0
        
        # Shift to prevent data leakage - look at previous games only
        shifted_series = series.shift(1)
        
        # Initialise streaks
        winning_streak = []
        losing_streak = []
        
        for i, won in enumerate(shifted_series):
            if pd.isna(won):  # First game has no history
                winning_streak.append(0)
                losing_streak.append(0)
                continue
                
            # Look backwards to count streak
            current_win_streak = 0
            current_loss_streak = 0
            
            # Count backwards from current position
            for j in range(i-1, -1, -1):
                if pd.isna(shifted_series.iloc[j]):
                    break
                    
                if shifted_series.iloc[j] == 1:  # Win
                    if current_loss_streak > 0:  # End of loss streak
                        break
                    current_win_streak += 1
                else:  # Loss
                    if current_win_streak > 0:  # End of win streak
                        break
                    current_loss_streak += 1
            
            winning_streak.append(current_win_streak)
            losing_streak.append(current_loss_streak)
        
        return pd.Series(winning_streak, index=series.index), pd.Series(losing_streak, index=series.index)
    
    # Apply streak calculation to each team
    streak_data = df.groupby('team_name')['won'].apply(get_current_streak)
    
    # Extract winning and losing streaks
    df['winning_streak'] = 0
    df['losing_streak'] = 0
    
    for team_name, (win_streaks, loss_streaks) in streak_data.items():
        team_mask = df['team_name'] == team_name
        df.loc[team_mask, 'winning_streak'] = win_streaks.values
        df.loc[team_mask, 'losing_streak'] = loss_streaks.values
    
    return df

team_stats_streaks = calculate_streaks(team_stats_form)

preview engineered features, might delete

In [ ]:
print("   Additional form indicators calculated")

# Data validation and summary
rolling_features = [col for col in team_stats_streaks.columns if col.startswith('rolling_')]
streak_features = [col for col in team_stats_streaks.columns if 'streak' in col]
form_features = [col for col in team_stats_streaks.columns if col.startswith(('recent_', 'games_since_'))]

all_new_features = rolling_features + streak_features + form_features

print(f"\n=== FEATURE ENGINEERING SUMMARY ===")
print(f" Rolling features created: {len(rolling_features)}")
print(f" Streak features created: {len(streak_features)}")
print(f" Form features created: {len(form_features)}")
print(f" Total new features: {len(all_new_features)}")

print(f"\nRolling features: {rolling_features}")
print(f"Streak features: {streak_features}")
print(f"Form features: {form_features}")

# Check for data leakage prevention
print(f"\n=== DATA LEAKAGE VALIDATION ===")

# Sort by team and date to get actual first games
df_sorted = team_stats_streaks.sort_values(['team_name', 'Date']).reset_index(drop=True)
first_games = df_sorted.groupby('team_name').first()

# Count null values in rolling features for first games
null_count = first_games[rolling_features].isnull().sum().sum()
total_first_games = len(first_games)
expected_nulls = total_first_games * len(rolling_features)

print(f"First game null values: {null_count}/{expected_nulls}")

# Additional validation: check if any first game has non-null rolling features
non_null_teams = []
for team in first_games.index:
    team_first_game = first_games.loc[team]
    if not team_first_game[rolling_features].isnull().all():
        non_null_teams.append(team)

if len(non_null_teams) == 0:
    print(f"Data leakage prevention:  PASS - All teams have null rolling features in first game")
else:
    print(f"Data leakage prevention:   PARTIAL - {len(non_null_teams)} teams have non-null values")
    print(f"  Teams with issues: {non_null_teams[:3]}...")  # Show first 3
    
    # Show example of what proper data leakage prevention looks like
    sample_team = df_sorted[df_sorted['team_name'] == first_games.index[0]].head(3)
    print(f"\n📊 Data Leakage Prevention Example ({first_games.index[0]}):")
    print("First 3 games should show: NaN → value → value pattern")
    print(f"rolling_avg_points_for_5: {sample_team['rolling_avg_points_for_5'].tolist()}")
    print("✅ This demonstrates proper .shift(1) behavior!")

## Step 4: Engineer Strength & Context Features

Beyond recent form, we need to capture inherent team strength and the context of the match. We engineer three key features:

### 4a. Elo Ratings
A dynamic rating system that measures a team's strength relative to its opponents over time. A win against a strong opponent yields more Elo points than a win against a weak one.

### 4b. Rest Days
Calculates the number of days a team has had to rest since their last match. This is a proxy for fatigue.

### 4c. Travel Distance
Calculates the distance an away team has to travel from their home city to the match venue. This is a proxy for travel fatigue.

In [ ]:
# All strength and context functions from your script would go here.
# For brevity in this example, I'll use placeholders, but you'd copy your full functions.
def calculate_elo_ratings(team_stats_df, k_factor=20, initial_elo=1500):
    """
    Step 4a: Calculate Elo ratings for each team
    
    Elo rating system tracks team strength over time based on match results.
    Higher Elo indicates stronger team. Ratings update after each match.
    
    Args:
        team_stats_df (pd.DataFrame): Team-level stats dataframe
        k_factor (int): K-factor for Elo rating changes (higher = more volatile)
        initial_elo (int): Starting Elo rating for all teams
        
    Returns:
        pd.DataFrame: Enhanced dataframe with pre-match Elo ratings
    """
    
    print("\n=== STEP 4a: Calculating Elo Ratings ===")
    
    # Initialize Elo ratings for all teams
    teams = team_stats_df['team_name'].unique()
    elo_ratings = {team: initial_elo for team in teams}
    
    print(f" Initialized {len(teams)} teams with Elo rating: {initial_elo}")
    
    # Create copy and sort by date
    df = team_stats_df.copy()
    df = df.sort_values(['Date', 'match_id']).reset_index(drop=True)
    
    # Add columns for pre-match Elo ratings
    df['pre_match_elo'] = 0.0
    
    # Process each match (two rows at a time - home and away)
    processed_matches = set()
    
    for idx, row in df.iterrows():
        match_id = row['match_id']
        
        # Skip if we've already processed this match
        if match_id in processed_matches:
            continue
        
        # Get both teams' records for this match
        match_data = df[df['match_id'] == match_id]
        
        if len(match_data) != 2:
            continue
            
        home_row = match_data[match_data['is_home'] == 1].iloc[0]
        away_row = match_data[match_data['is_home'] == 0].iloc[0]
        
        home_team = home_row['team_name']
        away_team = away_row['team_name']
        
        # Store pre-match Elo ratings
        home_pre_elo = elo_ratings[home_team]
        away_pre_elo = elo_ratings[away_team]
        
        # Update DataFrame with pre-match Elo
        df.loc[df['match_id'] == match_id, 'pre_match_elo'] = df.loc[df['match_id'] == match_id, 'team_name'].map({
            home_team: home_pre_elo,
            away_team: away_pre_elo
        })
        
        # Calculate expected scores using Elo formula
        # Home field advantage: add 100 Elo points to home team
        home_elo_adjusted = home_pre_elo + 100
        away_elo_adjusted = away_pre_elo
        
        expected_home = 1 / (1 + 10**((away_elo_adjusted - home_elo_adjusted) / 400))
        expected_away = 1 - expected_home
        
        # Actual results
        home_won = home_row['won']
        away_won = away_row['won']
        
        # Update Elo ratings
        elo_ratings[home_team] += k_factor * (home_won - expected_home)
        elo_ratings[away_team] += k_factor * (away_won - expected_away)
        
        processed_matches.add(match_id)
    
    print(f" Processed {len(processed_matches)} matches for Elo calculation")
    
    # Add final Elo ratings summary
    final_elos = pd.Series(elo_ratings).sort_values(ascending=False)
    print(f"\n=== ELO RATINGS SUMMARY ===")
    print("Top 5 teams by final Elo rating:")
    print(final_elos.head().to_string())
    print(f"\nBottom 5 teams by final Elo rating:")
    print(final_elos.tail().to_string())
    
    return df

def calculate_rest_days(df):
    print("\n--- Step 4b: Calculating Rest Days ---")
    df = df.sort_values(['team_name', 'Date'])
    df['rest_days'] = df.groupby('team_name')['Date'].diff().dt.days
    print("✓ Rest days calculated.")
    return df

def calculate_travel_distance(df):
    print("\n--- Step 4c: Calculating Travel Distance ---")
    # (Your haversine distance logic here)
    df['travel_distance_km'] = np.where(df['is_home'] == 0, 500, 0) # Placeholder
    print("✓ Travel distance calculated (using placeholder logic).")
    return df

# Execute Step 4 in sequence
team_stats_elo = calculate_elo_ratings(team_stats_streaks)
team_stats_rest = calculate_rest_days(team_stats_elo)
team_stats_final = calculate_travel_distance(team_stats_rest)

In [ ]:
def calculate_rest_days(team_stats_df):
    """
    Step 4b: Calculate rest days between matches for each team
    
    This function calculates rest days between consecutive matches within the same season.
    Off-season gaps (between Round 27 and Round 1 of next year) are excluded to prevent
    outliers from skewing the analysis.
    
    Args:
        team_stats_df (pd.DataFrame): Team-level stats dataframe
        
    Returns:
        pd.DataFrame: Enhanced dataframe with rest_days column
    """
    
    print("\n=== STEP 4b: Calculating Rest Days (Season-Aware) ===")
    
    df = team_stats_df.copy()
    df = df.sort_values(['team_name', 'Date']).reset_index(drop=True)
    
    # Convert Date to datetime if not already
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract year to identify seasons
    df['season'] = df['Date'].dt.year
    
    # Initialize rest_days column
    df['rest_days'] = np.nan
    
    print("Calculating rest days within seasons (excluding off-season gaps)...")
    
    # Calculate rest days for each team, respecting season boundaries
    for team in df['team_name'].unique():
        team_mask = df['team_name'] == team
        team_data = df[team_mask].copy()
        team_data = team_data.sort_values('Date').reset_index()
        
        # Calculate rest days between consecutive games
        for i in range(1, len(team_data)):
            current_season = team_data.iloc[i]['season']
            previous_season = team_data.iloc[i-1]['season']
            
            # Only calculate rest days within the same season
            if current_season == previous_season:
                current_date = team_data.iloc[i]['Date']
                previous_date = team_data.iloc[i-1]['Date']
                rest_days_value = (current_date - previous_date).days
                
                # Apply data leakage prevention by using previous game's rest days
                # (shift effect built into the logic)
                if i >= 2:  # Need at least 2 previous games to prevent leakage
                    df.loc[team_data.iloc[i]['index'], 'rest_days'] = rest_days_value
            # If different seasons, leave as NaN (off-season gap ignored)
    
    # Summary statistics (excluding NaN values)
    valid_rest_days = df['rest_days'].dropna()
    
    print(f" Rest days calculated for all teams (within seasons only)")
    print(f" Off-season gaps excluded: {df['rest_days'].isna().sum()} records")
    print(f" Valid rest day calculations: {len(valid_rest_days)} records")
    
    if len(valid_rest_days) > 0:
        rest_stats = valid_rest_days.describe()
        print(f"\n=== REST DAYS STATISTICS (Season-Aware) ===")
        print(f"Mean rest days: {rest_stats['mean']:.1f}")
        print(f"Median rest days: {rest_stats['50%']:.1f}")
        print(f"Min rest days: {rest_stats['min']:.0f}")
        print(f"Max rest days: {rest_stats['max']:.0f}")
        
        # Count of different rest periods
        rest_counts = valid_rest_days.value_counts().sort_index()
        print(f"\nMost common rest periods:")
        print(rest_counts.head(8).to_string())
        
        # Season analysis
        season_counts = df.groupby('season')['rest_days'].count()
        print(f"\n=== SEASON BREAKDOWN ===")
        print("Valid rest day calculations per season:")
        print(season_counts.to_string())
        
        # Check for any suspiciously long rest periods (potential issues)
        long_rest = valid_rest_days[valid_rest_days > 30]
        if len(long_rest) > 0:
            print(f"\n  Found {len(long_rest)} rest periods > 30 days:")
            long_rest_matches = df[df['rest_days'] > 30][['Date', 'team_name', 'rest_days', 'season']]
            print(long_rest_matches.head().to_string(index=False))
            print(f"Note: These may be mid-season breaks or scheduling anomalies")
        else:
            print(f"\n All rest periods are within reasonable range (≤30 days)")
    else:
        print(f"\n  No valid rest day calculations found")
    
    # Drop the temporary season column
    df = df.drop('season', axis=1)
    
    return df

In [ ]:
def calculate_travel_distance(team_stats_df):
    """
    Step 4c: Calculate travel distance for away teams
    
    Uses team home cities and match venues to calculate travel distance.
    Only away teams travel, so home teams get 0 distance.
    
    Args:
        team_stats_df (pd.DataFrame): Team-level stats dataframe
        
    Returns:
        pd.DataFrame: Enhanced dataframe with travel_distance_km column
    """
    
    print("\n=== STEP 4c: Calculating Travel Distance ===")
    
    # NRL team home cities (approximate coordinates)
    team_locations = {
        'Brisbane Broncos': (-27.4975, 153.0137),  # Brisbane
        'North Queensland Cowboys': (-19.2590, 146.8169),  # Townsville  
        'Gold Coast Titans': (-28.0167, 153.4000),  # Gold Coast
        'New Zealand Warriors': (-36.8485, 174.7633),  # Auckland
        'Melbourne Storm': (-37.8136, 144.9631),  # Melbourne
        'Canberra Raiders': (-35.2809, 149.1300),  # Canberra
        'Sydney Roosters': (-33.8688, 151.2093),  # Sydney
        'South Sydney Rabbitohs': (-33.8688, 151.2093),  # Sydney
        'St George Illawarra Dragons': (-34.4278, 150.8931),  # Wollongong
        'Cronulla-Sutherland Sharks': (-34.0544, 151.1518),  # Cronulla
        'Manly Sea Eagles': (-33.7969, 151.2841),  # Manly
        'Parramatta Eels': (-33.8176, 151.0032),  # Parramatta
        'Penrith Panthers': (-33.7506, 150.6934),  # Penrith
        'Wests Tigers': (-33.8688, 151.2093),  # Sydney
        'Canterbury Bulldogs': (-33.9173, 151.1851),  # Canterbury
        'Newcastle Knights': (-32.9283, 151.7817),  # Newcastle
        'Dolphins': (-27.4975, 153.0137),  # Brisbane (Redcliffe)
    }
    
    # Common venue locations
    venue_locations = {
        'Suncorp Stadium': (-27.4648, 153.0099),  # Brisbane
        'Queensland Country Bank Stadium': (-19.2598, 146.8181),  # Townsville
        'Cbus Super Stadium': (-28.0024, 153.3992),  # Gold Coast
        'AAMI Park': (-37.8255, 144.9816),  # Melbourne
        'GIO Stadium Canberra': (-35.2447, 149.1014),  # Canberra
        'Allianz Stadium': (-33.8878, 151.2273),  # Sydney
        'Accor Stadium': (-33.8474, 151.0616),  # Sydney Olympic Park
        'WIN Stadium': (-34.4056, 150.8841),  # Wollongong
        'PointsBet Stadium': (-34.0481, 151.1394),  # Cronulla
        '4 Pines Park': (-33.7742, 151.2606),  # Manly
        'CommBank Stadium': (-33.8007, 150.9810),  # Parramatta
        'BlueBet Stadium': (-33.7347, 150.6750),  # Penrith
        'Leichhardt Oval': (-33.8821, 151.1589),  # Leichhardt
        'McDonald Jones Stadium': (-32.9154, 151.7734),  # Newcastle
        'Mt Smart Stadium': (-36.9278, 174.8384),  # Auckland
        'Kayo Stadium': (-27.3644, 153.0486),  # Redcliffe
    }
    
    def haversine_distance(lat1, lon1, lat2, lon2):
        """Calculate distance between two points on Earth using Haversine formula"""
        from math import radians, sin, cos, sqrt, asin
        
        # Convert to radians
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        
        # Haversine formula
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        
        # Earth's radius in kilometers
        r = 6371
        
        return c * r
    
    df = team_stats_df.copy()
    
    # Initialize travel distance column
    df['travel_distance_km'] = 0.0
    
    # Calculate travel distance only for away teams
    away_games = df[df['is_home'] == 0].copy()
    
    for idx, row in away_games.iterrows():
        team_name = row['team_name']
        venue = row['Venue']
        
        # Get team home location
        if team_name in team_locations:
            team_lat, team_lon = team_locations[team_name]
        else:
            # Default to Sydney for unknown teams
            team_lat, team_lon = (-33.8688, 151.2093)
        
        # Get venue location (try exact match first, then partial match)
        venue_lat, venue_lon = None, None
        
        # Exact match
        if venue in venue_locations:
            venue_lat, venue_lon = venue_locations[venue]
        else:
            # Partial match for similar venue names
            for venue_key in venue_locations:
                if venue_key.lower() in venue.lower() or venue.lower() in venue_key.lower():
                    venue_lat, venue_lon = venue_locations[venue_key]
                    break
        
        # Default to team's home city if venue not found
        if venue_lat is None:
            venue_lat, venue_lon = team_lat, team_lon
        
        # Calculate distance
        distance = haversine_distance(team_lat, team_lon, venue_lat, venue_lon)
        df.loc[idx, 'travel_distance_km'] = distance
    
    print(f" Travel distances calculated for away games")
    
    # Summary statistics
    away_distances = df[df['is_home'] == 0]['travel_distance_km']
    travel_stats = away_distances.describe()
    
    print(f"\n=== TRAVEL DISTANCE STATISTICS ===")
    print(f"Mean travel distance: {travel_stats['mean']:.1f} km")
    print(f"Median travel distance: {travel_stats['50%']:.1f} km")
    print(f"Max travel distance: {travel_stats['max']:.1f} km")
    print(f"Min travel distance: {travel_stats['min']:.1f} km")
    
    # Show longest travels
    longest_travels = df[df['travel_distance_km'] > 0].nlargest(5, 'travel_distance_km')
    print(f"\nLongest travel distances:")
    travel_display = longest_travels[['Date', 'team_name', 'Venue', 'City', 'travel_distance_km']]
    print(travel_display.to_string(index=False))
    
    return df

## Step 5: Assemble Final Model-Ready DataFrame

This is the final and most important step. We merge all the engineered team-level features back into our original match-level DataFrame.

The key action here is creating **difference features** (e.g., `elo_diff`, `form_margin_diff_5`). Our model will learn from the *relative difference* in form and strength between the two competing teams, which is much more predictive than looking at each team's stats in isolation.

In [ ]:
def assemble_final_model_ready_dataframe(df_matches, df_team_features):
    print("\n--- Step 5: Assembling Final Model-Ready DataFrame ---")
    
    # Split features into home and away sets
    home_stats = df_team_features[df_team_features['is_home'] == 1].copy()
    away_stats = df_team_features[df_team_features['is_home'] == 0].copy()

    # Select and rename feature columns
    feature_cols = [col for col in df_team_features.columns if col.startswith(('rolling', 'pre_match', 'rest', 'travel'))]
    home_features = home_stats[['match_id'] + feature_cols].add_prefix('home_')
    away_features = away_stats[['match_id'] + feature_cols].add_prefix('away_')

    # Merge back to the match dataframe
    df_final = df_matches.merge(home_features, left_on='match_id', right_on='home_match_id', how='left')
    df_final = df_final.merge(away_features, left_on='match_id', right_on='away_match_id', how='left')

    # Create difference features
    df_final['elo_diff'] = df_final['home_pre_match_elo'] - df_final['away_pre_match_elo']
    for window in [3, 5, 8]:
        df_final[f'form_margin_diff_{window}'] = df_final[f'home_rolling_avg_margin_{window}'] - df_final[f'away_rolling_avg_margin_{window}']
        df_final[f'form_win_rate_diff_{window}'] = df_final[f'home_rolling_win_percentage_{window}'] - df_final[f'away_rolling_win_percentage_{window}']
        # ... (add other diffs as needed)
    
    # Add market features
    df_final['home_implied_prob'] = 1 / df_final['Home Odds']
    df_final['away_implied_prob'] = 1 / df_final['Away Odds']
    df_final['market_spread'] = df_final['home_implied_prob'] - df_final['away_implied_prob']
    
    print(f"✓ Final model-ready DataFrame created. Shape: {df_final.shape}")
    return df_final

# Execute Step 5
df_model_ready = assemble_final_model_ready_dataframe(df_cleaned, team_stats_final)

## Step 6: Save Final Datasets

The pipeline is complete. We save the final model-ready DataFrame to a new CSV file. This file will be the single source of truth for all subsequent model training and evaluation.

In [ ]:
print("\n--- Step 6: Saving Final Datasets ---")

output_path = 'data/nrl_matches_final_model_ready.csv'
df_model_ready.to_csv(output_path, index=False)

print(f"✓ Final dataset saved successfully to: {output_path}")
print("\n🏆 Pipeline Complete! The data is now ready for model training. 🏆")